In [ ]:
import numpy as np
from scipy.optimize import minimize, LinearConstraint, Bounds
from itertools import *
from tqdm.notebook import trange, tqdm
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

def powerset(iterable):
    "powerset([1,2,3]) → () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [ ]:
r = 6

In [ ]:
A = np.zeros((4, 3 * r))
A[0, 0:r] = np.arange(1, r + 1)
A[1, r:2*r] = np.arange(1, r + 1)
A[2, 2*r:3*r] = np.arange(1, r + 1)
A[3] = 1
A

array([[1., 2., 3., 4., 5., 6., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 2., 3., 4., 5., 6., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 2., 3., 4.,
        5., 6.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.]])

In [ ]:
B = np.zeros((3, 3 * r))
B[0, 0:r] = 1
B[1, r:2*r] = 1
B[2, 2*r:3*r] = 1
B

array([[1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
        1., 1.]])

In [ ]:
def trivial(X, lam):
    return lam - np.max(B.dot(X))

In [ ]:
def fourier(X, lam):
    tot_ab = 0
    max_ab = 0
    tot_ac = 0
    max_ac = 0
    tot_bc = 0
    max_bc = 0
    for i in range(r):
        a = X[i]
        b = X[r + i]
        c = X[2*r + i]
        ab = max(a, b)
        ac = max(a, c)
        bc = max(b, c)
        tot_ab += ab
        tot_ac += ac
        tot_bc += bc
        max_ab = max(max_ab, ab)
        max_ac = max(max_ac, ac)
        max_bc = max(max_bc, bc)
    val_ab = tot_ab - max_ab
    val_ac = tot_ac - max_ac
    val_bc = tot_bc - max_bc
    val = min(val_ab, val_ac, val_bc)
    return (val + lam) / 2

In [ ]:
intervals = []

for a in range(r):
    aI = range(a + 1)
    intervals.append(aI)

intervals = [np.array(i, dtype=int) for i in intervals]

In [ ]:
def basic_geometry(X, lam):
    best = 3
    for aJ, bJ, cJ in product(intervals, repeat=3):
        I = np.hstack([aJ, bJ, cJ])
        J = np.hstack([aJ, bJ + r, cJ + 2 * r])
        val1 = max(1, np.sum(X[J] * (I + 1)))
        val2 = np.sum(X[J])
        best = min(best, val1 - val2)
    return lam - 1 + best

In [ ]:
def geometry(X, lam):
    best = 3
    for aI, bI, cI in product(powerset(range(r)), repeat=3):
        aJ = np.array(aI, dtype=int)
        bJ = np.array(bI, dtype=int)
        cJ = np.array(cI, dtype=int)
        I = np.hstack([aJ, bJ, cJ])
        J = np.hstack([aJ, bJ + r, cJ + 2 * r])
        val1 = max(1, np.sum(X[J] * (I + 1)))
        val2 = np.sum(X[J])
        best = min(best, val1 - val2)
    return lam - 1 + best

In [ ]:
def geometry_careful(X, lam):
    best = 3
    best_arg = None
    for aI, bI, cI in product(powerset(range(r)), repeat=3):
        aJ = np.array(aI, dtype=int)
        bJ = np.array(bI, dtype=int)
        cJ = np.array(cI, dtype=int)
        I = np.hstack([aJ, bJ, cJ])
        J = np.hstack([aJ, bJ + r, cJ + 2 * r])
        val1 = max(1, np.sum(X[J] * (I + 1)))
        val2 = np.sum(X[J])
        if val1 - val2 < best:
            best = val1 - val2
            best_arg = (aI, bI, cI)
    print(best_arg)
    return lam - 1 + best

In [ ]:
def thue(X, lam):
    tot_ab = tot_ac = tot_bc = 0
    for p in range(2, r + 1):
        val_a = val_b = val_c = 0
        for i in range(p, r + 1, p):
            val_a += X[i - 1]
            val_b += X[r + i - 1]
            val_c += X[2 * r + i - 1]
        tot_ab = max(tot_ab, val_a + val_b)
        tot_ac = max(tot_ac, val_a + val_c)
        tot_bc = max(tot_bc, val_b + val_c)
    return min(lam - tot_ab, lam - tot_ac, lam - tot_bc)

In [ ]:
def determinant(X, lam):
    result = 10
    shifts = [(0, r), (0, 2 * r), (r, 2 * r)]
    for x, y in shifts:
        for p in range(1, r + 1):
            for q in range(1, r + 1):
                ap = X[x + p - 1]
                bq = X[y + q - 1]
                result = min(result, lam - ap - bq + min(ap / q, bq / p))
    return result

In [ ]:
def constraints(lam):
    return [LinearConstraint(A,
                    lb=np.array([-np.inf, -np.inf, 1, -np.inf]),
                    ub=np.array([1, 1, 1, lam]))]

In [ ]:
def combined(X, lam):
    return -min(trivial(X, lam), fourier(X, lam), basic_geometry(X, lam), thue(X, lam), determinant(X, lam))

In [ ]:
def optimise(X, lam):
    sol = minimize(combined, x0=X,
                   constraints=constraints(lam),
                #    method='trust-constr',
                   bounds=Bounds(lb=0),
                   args=(lam,))
    return sol

In [ ]:
solutions = {i/20: (-2, None) for i in range(41)}
vals = solutions.keys()

In [ ]:
for _ in range(50):
    for lam in vals:
        if lam <= 1.5: continue
        X = np.random.rand(3 * r)
        sol = optimise(X, lam)
        if not sol.success:
            print(f"{lam} failed")
            continue
        X = sol.x
        nu = min(trivial(X, lam), fourier(X, lam), geometry(X, lam), thue(X, lam), determinant(X, lam))

        print(f"{lam:.2f}", f"{-sol.fun:.6f}", f"{nu:.6f}")
        if abs(-sol.fun - nu) > 0.01:
            geometry_careful(X, lam)
        # if abs(nu - (lam - 1)) < 1e-6:
        #     print(sol)
        #     print(np.array2string(np.reshape(X, (3, r)), separator=', ', precision=6, formatter=fmt))

        if nu > solutions[lam][0]:
            solutions[lam] = (nu, X)

            print('improvement!')

    for i in vals:
        (j, k) = solutions[i]
        if k is not None:
            print(f"{j:.6f}")
    # for i in vals:
    #     (j, k) = solutions[i]
    #     if k is not None:
    #         print(f"{i:.2f}: ({j:.6f},\n",
    #             f"np.array({np.array2string(np.reshape(k, (3, r)), separator=', ', precision=6, formatter=fmt)})),"
    #             )

1.55 1.006214 0.981984
((0, 1, 3, 4, 5), (0, 1, 4, 5), (0, 1, 2, 3, 5))
1.60 1.039499 1.036261
improvement!
1.65 1.012628 0.932879
((0, 1, 2, 3, 5), (0, 2, 3, 4, 5), (0, 1, 3, 4, 5))
1.70 1.096300 1.077307
((0,), (0, 4), (0,))
1.75 1.104124 1.083691
((0, 1, 2, 4, 5), (0, 1, 3, 4), (0, 1, 4, 5))
1.80 0.800000 0.800000
1.85 1.176907 1.160642
((0, 2, 4, 5), (0, 1, 3, 4, 5), (0, 1, 3, 5))
1.90 1.193135 1.193135
1.95 0.950000 0.950000
2.00 1.257581 1.257581
0.033333
0.083333
0.133333
0.175000
0.200000
0.239467
0.288193
0.326353
0.370714
0.401264
0.436739
0.479409
0.516517
0.551225
0.590603
0.616099
0.642362
0.684620
0.708877
0.750572
0.780021
0.812964
0.845094
0.871958
0.912021
0.940107
0.975732
0.999034
1.036261
1.067171
1.080287
1.125717
1.152470
1.180547
1.222682
1.224930
1.271461
1.55 0.962860 0.934439
((), (0, 1), (0, 1, 4))
1.60 1.027101 0.992296
((0, 1, 3, 4, 5), (0, 2, 3, 4, 5), (0, 1, 2, 3, 5))
1.65 1.043146 1.043146
1.70 1.067591 1.067591
1.75 0.750000 0.750000
1.80 0.800000 0.800

In [ ]:
print(f"lambda: {lam}")
print(f"vectors:\n{np.array2string(np.reshape(X, (3, r)), separator=', ')}")
print(f"approx_nu:      {-sol.fun:.6f}")
print(f"trivial:        {trivial(X, lam):.6f}")
print(f"fourier:        {fourier(X, lam):.6f}")
print(f"basic_geometry: {basic_geometry(X, lam):.6f}")
print(f"geometry:       {geometry(X, lam):.6f}")
print(f"thue:           {thue(X, lam):.6f}")
print(f"determinant:    {determinant(X, lam):.6f}")
nu = min(trivial(X, lam), fourier(X, lam), geometry(X, lam), thue(X, lam), determinant(X, lam))
print(f"nu:             {nu:.6f}")


# for i in [trivial(X), fourier(X), basic_geometry(X), geometry(X), thue(X), determinant(X)]:
#     print(f"{i:.6f}")

In [ ]:
fmt = {'float_kind': lambda x: f"{x:.6f}"}

In [ ]:
for i in vals:
    (j, k) = solutions[i]
    if k is not None:
        print(f"{i:.2f}: ({j:.6f},\n",
            f"np.array({np.array2string(np.reshape(k, (3, r)), separator=', ', precision=6, formatter=fmt)})),"
            )

0.20: (0.033333,
 np.array([[0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000],
 [0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000],
 [0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.166667]])),
0.25: (0.083333,
 np.array([[0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000],
 [0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000],
 [0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.166667]])),
0.30: (0.133333,
 np.array([[0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000],
 [0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000],
 [0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.166667]])),
0.35: (0.175000,
 np.array([[0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000],
 [0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000],
 [0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.166667]])),
0.40: (0.200000,
 np.array([[0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000],
 [0.000000, 0.000000, 0.000000, 0.